In [ ]:
from IPython.display import Audio, display
import ipywidgets as widgets
import librosa
import numpy as np
from tensorflow.keras.models import load_model
import io
!pip install pydub==0.25.1
import pydub

# Load your trained model
model = load_model('/content/drive/MyDrive/BIA/lstm_model.h5')

# Define a function to extract features from audio
def extract_features_mfcc(audio_data, sr):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=40)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs

# Define a function to make predictions
def predict_emotion(audio_file):
    try:
        # Attempt to load with librosa (for .wav)
        audio_data, sr = librosa.load(audio_file)
    except:
        # If librosa fails, try loading as .mp3 using pydub
        audio = pydub.AudioSegment.from_file(audio_file, format="mp3")
        audio_data = np.array(audio.get_array_of_samples())
        sr = audio.frame_rate

    features = extract_features_mfcc(audio_data, sr)
    features = np.expand_dims(features, axis=0)  # Add batch dimension
    features = np.expand_dims(features, axis=-1)
    prediction = model.predict(features)
    emotion_labels = ['angry', 'disgusting', 'fear', 'happy', 'neutral', 'sad']  # Adjust labels if needed
    predicted_emotion = emotion_labels[np.argmax(prediction)]
    return predicted_emotion



# Widget for uploading audio file
uploader = widgets.FileUpload(accept='.wav, .mp3', multiple=False)  # Accept both .wav and .mp3
display(uploader)


# Function to handle file upload and prediction
def on_upload_change(change):
    if change['new']:
        uploaded_file = list(uploader.value.values())[0]

        # Load audio data, handling .mp3 using pydub if necessary
        try:
            # Attempt to load with librosa (for .wav)
            audio_data, sr = librosa.load(io.BytesIO(uploaded_file['content']))
        except:
            # If librosa fails, try loading as .mp3 using pydub
            audio = pydub.AudioSegment.from_file(io.BytesIO(uploaded_file['content']), format="mp3")
            audio_data = np.array(audio.get_array_of_samples())
            sr = audio.frame_rate

        # Display audio
        display(Audio(data=audio_data, rate=sr))

        # Make prediction
        predicted_emotion = predict_emotion(io.BytesIO(uploaded_file['content']))  # Pass BytesIO for .mp3
        print(f"Predicted Emotion: {predicted_emotion}")

# Observe changes in the uploader widget
uploader.observe(on_upload_change, names='value')

FileUpload(value={}, accept='.wav, .mp3', description='Upload')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
Predicted Emotion: happy
